In [1]:
from Utils_MSA import *
from importlib import reload
    
rcParams = {'font.size': 24, 'font.weight': 'normal', 'font.family': 'sans-serif',
            'axes.unicode_minus':False, 'axes.labelweight':'normal'}
plt.rcParams.update(rcParams)

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [2]:
filedir = '/Users/harihara/Mount-2/hotspring_metagenome/single_cell_analysis_with_Gabe_Birzu/16S_Read_Alignments/'
samples = listdir(filedir+'Alpha_Alleles/paf_files/')

fdir = '/Users/harihara/Mount-2/hotspring_metagenome/single_cell_analysis_with_Gabe_Birzu/16S_Gene_Segments/'
out = {}
for f in ['Synechococcus_Alpha_16SrRNA','Synechococcus_Beta_16SrRNA','Synechococcus_Gamma_16SrRNA']:
    lines = open(fdir+f+'.fna').readlines()
    s = ''
    for l in lines:
        if not l.startswith('>'):
            s += l.replace("\n","")
    out[f.replace("Synechococcus_","").replace("_16SrRNA","")] = s    

In [64]:
start = 350
end = 550
MSA = {'Alpha':{'Alpha_Ref':out['Alpha'][start:end]}, 'Beta':{'Beta_Ref':out['Beta'][start:end]}, 
       'Gamma':{'Gamma_Ref':out['Gamma'][start:end]}}

MSA_duplicates = {'Alpha':dict({}), 'Beta':dict({}), 'Gamma':dict({})}
df_op = pd.DataFrame()
retain_duplicates = False

for s in samples:
    if 'MS65' not in s:
        continue
    print(s)
    
    df_alpha = Load_PAF(filedir+'Alpha_Alleles/paf_files/'+s, s)
    df_beta = Load_PAF(filedir+'Beta_Alleles/paf_files/'+s, s)
    df_gamma = Load_PAF(filedir+'Gamma_Alleles/paf_files/'+s, s)
    
    Temp = pd.DataFrame()
    Temp = Temp.append(df_alpha).append(df_beta).append(df_gamma)
    df_op = df_op.append(Temp, ignore_index = True)
    
    D = Create_MSA(Temp, start, end, s, out)
    print(MSA.keys())
    if retain_duplicates == False:
        for a in MSA.keys():
            try:
                MSA[a].update(D[a])
            except KeyError:
                print('Key Missing', a)
    else:
        for a in D.keys():
            for o in D[a]:
                if D[a][o] not in MSA_duplicates[a]:
                    MSA[a][o] = D[a][o]
                    MSA_duplicates[a][D[a][o]] = [o]
                else:
                    MSA_duplicates[a][D[a][o]].append(o)  
            print(a, len(MSA_duplicates[a]), len(MSA[a]))
          

HotsprSampleMS65_FD.paf
421 43206
263 42121
2084 49871
dict_keys(['Alpha', 'Beta', 'Gamma'])


In [65]:
alpha = MSA['Alpha']['Alpha_Ref']
beta = MSA['Beta']['Beta_Ref']
gamma = MSA['Gamma']['Gamma_Ref']

del MSA['Alpha']['Alpha_Ref'],MSA['Beta']['Beta_Ref'],MSA['Gamma']['Gamma_Ref']

In [66]:
d_alpha_beta, tracker_alpha_beta = Edit(alpha, beta)
Align_alpha, Align_beta = Return_Alignments(tracker_alpha_beta, alpha, beta)

d_alpha_beta_gamma, tracker_alpha_beta_gamma = Edit(Align_alpha, gamma) 
Align_alpha_gamma, Align_gamma = Return_Alignments(tracker_alpha_beta_gamma, Align_alpha, gamma, '+')
Align_beta_gamma = Merge_MSA(Align_alpha_gamma, {'Beta':Align_beta}, '+')['Beta']

Alpha_backbone = Align_alpha_gamma.replace("+","-")
Beta_backbone = Align_beta_gamma.replace("+","-")
Gamma_backbone = Align_gamma.replace("+","-")

assert ((len(Alpha_backbone) == len(Beta_backbone)) & 
        (len(Gamma_backbone) == len(Beta_backbone))), "Length Mismatch"

In [67]:
MSA_alpha = Merge_MSA(Alpha_backbone, MSA['Alpha'], '-')
MSA_beta = Merge_MSA(Beta_backbone, MSA['Beta'], '-')
MSA_gamma = Merge_MSA(Gamma_backbone, MSA['Gamma'], '-')

MSA_All = MSA_alpha.copy()
MSA_All.update(MSA_beta)
MSA_All.update(MSA_gamma)

MSA_All['Alpha'] = Alpha_backbone
MSA_All['Beta'] = Beta_backbone
MSA_All['Gamma'] = Gamma_backbone

print(len(MSA_All))

1990


In [68]:
o = open('/Users/harihara/Mount-2/hotspring_metagenome/single_cell_analysis_with_Gabe_Birzu/\
16S_Read_Alignments/MSA_MS65_'+str(start)+'_'+str(end)+'.fasta','w')
for M in MSA_All:
    o.write(">"+M+"\n")
    o.write(MSA_All[M]+"\n")
o.close()

In [19]:
o = open('/Users/harihara/Mount-2/hotspring_metagenome/single_cell_analysis_with_Gabe_Birzu/\
16S_Read_Alignments/MSA_'+str(start)+'_'+str(end)+'_Duplicates.dict','w')
o.write(str(MSA_duplicates))
o.close()

In [28]:
MSA_duplicates

{'Alpha': {}, 'Beta': {}, 'Gamma': {}}